In [1]:
from src.scraping.google_scraping.review_data_saver import ReviewDataSaver
from src.scraping.google_scraping.review_extractor import ReviewExtractor
from src.scraping.google_scraping.review_loader import ReviewLoader
from concurrent.futures import ThreadPoolExecutor, as_completed
from src.scraping.browser_managment.browser_manager import BrowserManager  # Ensure this is the updated BrowserManager
from src.scraping.browser_managment.parallel_browser_manager import ParallelBrowserManager
from selenium.webdriver.common.by import By
import time
from src.scraping.google_scraping import RestaurantListScraper
from src.scraping.google_scraping.data_saver import DataSaver

from src.scraping.google_scraping.review_interaction_handler import ReviewInteractionHandler
city='Zagreb'
query=f'Restorani {city}'


browser_manager = BrowserManager()
review_handler = ReviewInteractionHandler(browser_manager.driver)
browser_manager.navigate_to_url(f"https://www.google.hr/search?q={query}")
review_handler.accept_cookies()

The chromedriver version (121.0.6167.85) detected in PATH at C:\Users\ivan.zeljeznjak\Desktop\chromedriver-win64\chromedriver.exe might not be compatible with the detected chrome version (122.0.6261.112); currently, chromedriver 122.0.6261.111 is recommended for chrome 122.*, so it is advised to delete the driver in PATH and retry


In [2]:

review_handler.navigate_to_restaurant_list()


scraper = RestaurantListScraper(browser_manager.driver)
# Assume driver has navigated to the restaurant list page
df_restaurants = scraper.scrape_restaurants()

data_saver = DataSaver(df_restaurants, city)
data_saver.save_to_csv()
time.sleep(5)
browser_manager.close_browser()

No new content loaded. End of list reached.
New restaurants to add:
- Vinska Klet
- KAVKAZ kazališna kavana
- Gostionica Purger
- Pizzeria Fianona
- OAZA Joyful Kitchen
- Pilana Bliznec
- HERITAGE - Croatian Street Food & Shop
- La Štruk
- Ž𝒊𝒓𝒂𝒇𝒇𝒂 𝑭𝒐𝒐𝒅
- Al Dente Zagreb
- Zrno bio bistro
- Restoran "Zlatna školjka"
- Pizzeria Purger
- Asia Gold Cuisine
- Leonardo trattoria & pizzeria
- Shambala
- Lanterna na Dolcu
- Konoba Primošten
- Restoran Nokturno
- Restoran Baltazar
- Restoran Tač
- Restoran Sv. Nikola
- Boogie Lab Zagreb
- Pivnica Mali Medo
- Restoran Zinfandel's
- Bistro&Pizzeria Public
- O'brok Restoran
- Cunami
- Roots - juice & cocktail bar
- Restoran Bicko
- Restoran Bonaca
- Health n joy
- SILK - Pan-Asian Fresh Street Food - Zagreb
- Bistro Fotić
- Otto & Frank
- Restaurant and Caffe "Muzej"
- Curry Bowl
- Bistro Jarun
- R&B Food
- Torero Restaurant
- Le Bistro
- Restoran Sir Oliver
- Restaurant & Bar Frida
- Nico's Bar & Bistro
- River pub
- Mano2
- Sakura Sushi Restaura

In [3]:
df_restaurants


,Restaurant Name
0,Vinska Klet
1,KAVKAZ kazališna kavana
2,Gostionica Purger
3,Pizzeria Fianona
4,OAZA Joyful Kitchen
...,...
214,Bistro Apetit by Marin Rendić
215,Bistro Fetiš - gluten free food
216,Gajbica
217,Ukusno i .


In [4]:
import pandas as pd

# Define the path to the CSV file
file_path = r'C:\Users\ivan.zeljeznjak\DataspellProjects\RedditSentimentAnalysis\data\Zagreb\restaurant_list.csv'

# Load the CSV file into a DataFrame
df = pd.read_csv(file_path)

# Display the DataFrame
df


,Restaurant Name
0,Vinska Klet
1,KAVKAZ kazališna kavana
2,Gostionica Purger
3,Pizzeria Fianona
4,OAZA Joyful Kitchen
...,...
214,Bistro Apetit by Marin Rendić
215,Bistro Fetiš - gluten free food
216,Gajbica
217,Ukusno i .


In [17]:
from datetime import datetime
from src.scraping.google_scraping.review_data_saver import ReviewDataSaver
from src.scraping.google_scraping.review_extractor import ReviewExtractor
from src.scraping.google_scraping.review_loader import ReviewLoader
from concurrent.futures import ThreadPoolExecutor, as_completed
from src.scraping.browser_managment.browser_manager import BrowserManager  # Ensure this is the updated BrowserManager
from src.scraping.browser_managment.parallel_browser_manager import ParallelBrowserManager
from selenium.webdriver.common.by import By
import time
from src.scraping.google_scraping.review_interaction_handler import ReviewInteractionHandler

def perform_search(query):
    browser_manager = BrowserManager()
    review_handler = ReviewInteractionHandler(browser_manager.driver)
    review_loader = ReviewLoader(browser_manager.driver)
    review_extractor = ReviewExtractor(browser_manager.driver)
    attempt_status = 'Failed'  # Default to 'Failed', update to 'Success' if no exceptions occur
    attempt_date = datetime.now().strftime("%Y-%m-%d %H:%M")
    loaded_reviews = 0
    scraped_reviews = 0
    percentage = 0.0
    
    try:
        browser_manager.navigate_to_url(f"https://www.google.hr/search?q={query}")
        review_handler.accept_cookies()
        review_handler.navigate_to_reviews()
        review_loader.load_reviews()  # Load all reviews
        review_handler.click_more_buttons()  # Click all "More" buttons
        reviews_df = review_extractor.collect_reviews()  # Adjust based on actual return type/method
        attempt_status = 'Success'

        # Success metrics calculations and prints can remain here or be moved as appropriate
        loaded_reviews = review_loader.total_reviews_loaded
        scraped_reviews = review_extractor.total_scraped_reviews
        ratio = scraped_reviews / loaded_reviews if loaded_reviews > 0 else 0
        percentage = (scraped_reviews / loaded_reviews * 100) if loaded_reviews > 0 else 0

        print(f"Query: {query}")
        print(f"Final count of reviews loaded: {loaded_reviews}, Scraped/Loaded Ratio: {scraped_reviews}/{loaded_reviews} = {ratio:.2f}, Success Percentage: {percentage:.2f}%")
        review_data_saver = ReviewDataSaver(reviews_df, query)
        review_data_saver.save_reviews_to_csv()
    except Exception as e:
        print(f"An error occurred during search for '{query}': {e}")
        attempt_status = 'Failed'
    finally:
        browser_manager.close_browser()

    return query, attempt_status, attempt_date, loaded_reviews, scraped_reviews, percentage



In [ ]:
import pandas as pd
from datetime import datetime, timedelta
import numpy as np
import os
# Ensure correct imports for your scraping and management classes
from src.scraping.browser_managment.parallel_browser_manager import ParallelBrowserManager


# Define paths and initial setup
city = 'Zagreb'
base_path = '../data'
directory_path = os.path.join(base_path, city)
csv_file_path = os.path.join(directory_path, 'restaurant_list.csv')
os.makedirs(directory_path, exist_ok=True)

# Read or initialize the DataFrame
if os.path.exists(csv_file_path):
    df = pd.read_csv(csv_file_path)
else:
    df = pd.DataFrame()

# Ensure all required columns exist
required_columns = ['Restaurant Name', 'Last scraping attempt', 'Last scraping attempt date',
                    'Loaded Reviews', 'Scraped Reviews', 'Scraping Success Percentage']
for column in required_columns:
    if column not in df.columns:
        if 'date' in column:
            df[column] = pd.NaT
        else:
            df[column] = np.nan

# Define scraping conditions
one_month_ago = datetime.now() - timedelta(days=30)
should_scrape_conditions = (
        df['Last scraping attempt'].isna() |
        (df['Last scraping attempt'] == 'Not attempted') |
        (df['Last scraping attempt'] == 'Failed') |
        df['Last scraping attempt date'].isna() |
        (df['Last scraping attempt date'] < one_month_ago) |
        df['Scraping Success Percentage'].isna()
)

# Generate scraping queries
queries = [f"{row['Restaurant Name']} {city}" for _, row in df[should_scrape_conditions].iterrows()]

# Initialize ParallelBrowserManager with required arguments
parallel_manager = ParallelBrowserManager(df, csv_file_path, city, workers=3)

# Prepare the list of arguments for each task
args_list = [(query,) for query in queries]

# Perform the scraping in parallel and update the DataFrame
parallel_manager.perform_parallel_action(perform_search, args_list)

# Save the updated DataFrame
df.to_csv(csv_file_path, index=False)


In [2]:
#placeholder